In [1]:
import requests
import time
import json

GITHUB_API_URL = "https://api.github.com"

def create_headers(access_token):
    return {
        "Accept": "application/vnd.github+json",
        "Authorization": f"Bearer {access_token}",
        "X-GitHub-Api-Version": "2022-11-28"
    }

def handle_rate_limit(response):
    if response.status_code == 403 and 'X-RateLimit-Remaining' in response.headers:
        if response.headers.get('X-RateLimit-Remaining') == '0':
            reset_time = int(response.headers.get('X-RateLimit-Reset', time.time() + 60))
            wait_time = max(reset_time - time.time(), 1)
            print(f"[Rate Limit] Exceeded. Waiting {wait_time} seconds until reset.")
            time.sleep(wait_time)
            return True
    return False

def search_repositories(query, access_token, per_page=30, max_pages=5):
    print(f"Searching repositories with query: {query}")
    headers = create_headers(access_token)
    all_results = []
    base_url = f"{GITHUB_API_URL}/search/repositories"

    for page in range(1, max_pages + 1):
        params = {"q": query, "per_page": per_page, "page": page}
        response = requests.get(base_url, headers=headers, params=params)

        if handle_rate_limit(response):
            continue

        if response.status_code == 200:
            data = response.json().get('items', [])
            if not data:
                print(f"[Search] End of results on page {page}.")
                break
            all_results.extend(data)
            print(f"[Search] Page {page}: {len(data)} repositories found.")
        else:
            print(f"[Search] Error {response.status_code}: {response.text}")
            break

        time.sleep(1)

    print(f"[Search] Total repositories found: {len(all_results)}")
    return all_results

def list_commits(owner, repo, access_token, per_page=30, max_pages=5):
    print(f"Listing commits for repository: {owner}/{repo}")
    headers = create_headers(access_token)
    all_commits = []
    base_url = f"{GITHUB_API_URL}/repos/{owner}/{repo}/commits"

    for page in range(1, max_pages + 1):
        params = {"per_page": per_page, "page": page}
        response = requests.get(base_url, headers=headers, params=params)

        if handle_rate_limit(response):
            continue

        if response.status_code == 200:
            commits = response.json()
            if not commits:
                print(f"[Commits] End of commits on page {page}.")
                break
            all_commits.extend(commits)
            print(f"[Commits] Page {page}: {len(commits)} commits found.")
        else:
            print(f"[Commits] Error {response.status_code}: {response.text}")
            break

        time.sleep(1)

    print(f"[Commits] Total commits found: {len(all_commits)}")
    return all_commits

def get_contents(owner, repo, access_token, path=""):
    print(f"Reading contents from: {owner}/{repo}/{path if path else '[root]'}")
    headers = create_headers(access_token)
    base_url = f"{GITHUB_API_URL}/repos/{owner}/{repo}/contents/{path}"
    all_files = []

    try:
        response = requests.get(base_url, headers=headers)

        if handle_rate_limit(response):
            return get_contents(owner, repo, access_token, path)

        if response.status_code == 200:
            contents = response.json()

            if isinstance(contents, list):
                for item in contents:
                    all_files.append(item)
                print(f"[Contents] {len(contents)} items found in directory '{path or 'root'}'.")
            else:
                all_files.append(contents)
                print(f"[Contents] Single file found: {contents.get('name')}")
        else:
            print(f"[Contents] Error {response.status_code}: {response.text}")

    except Exception as e:
        print(f"[Contents] Exception while fetching contents: {e}")

    return all_files

In [3]:
if __name__ == "__main__":
    ACCESS_TOKEN = ""

    repos = search_repositories("language:python", ACCESS_TOKEN, per_page=10, max_pages=2)
    commits = list_commits("victormartorelli", "my-scripts", ACCESS_TOKEN, per_page=10, max_pages=2)
    contents = get_contents("victormartorelli", "my-scripts", ACCESS_TOKEN)

    print(repos)
    print(commits)
    print(contents)

Searching repositories with query: language:python
[Search] Page 1: 10 repositories found.
[Search] Page 2: 10 repositories found.
[Search] Total repositories found: 20
Listing commits for repository: victormartorelli/my-scripts
[Commits] Page 1: 10 commits found.
[Commits] Page 2: 7 commits found.
[Commits] Total commits found: 17
Reading contents from: victormartorelli/my-scripts/[root]
[Contents] 7 items found in directory 'root'.
[{'id': 13491895, 'node_id': 'MDEwOlJlcG9zaXRvcnkxMzQ5MTg5NQ==', 'name': 'free-programming-books', 'full_name': 'EbookFoundation/free-programming-books', 'private': False, 'owner': {'login': 'EbookFoundation', 'id': 14127308, 'node_id': 'MDEyOk9yZ2FuaXphdGlvbjE0MTI3MzA4', 'avatar_url': 'https://avatars.githubusercontent.com/u/14127308?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/EbookFoundation', 'html_url': 'https://github.com/EbookFoundation', 'followers_url': 'https://api.github.com/users/EbookFoundation/followers', 'following_url': 'ht